In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from helper_functions import accuracy_fn
from sklearn.preprocessing import StandardScaler

### Data Processing

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
train_df = pd.read_csv("sign_mnist_train.csv")
test_df = pd.read_csv("sign_mnist_test.csv")

In [4]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, feature_columns, target_column=None):
        self.dataframe = dataframe
        self.feature_columns = feature_columns
        self.target_column = target_column

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Convert features to a tensor
        features = torch.tensor(self.normalize(self.dataframe.iloc[idx][self.feature_columns].values), dtype=torch.float32, device=device).reshape((1, 28, 28))
        
        # If there's a target column, return it as well
        if self.target_column is not None:
            target = torch.tensor(self.dataframe.iloc[idx][self.target_column]).type(torch.LongTensor).to(device)
            return features, target
        
        return features
    
    def normalize(self, table):
        scaler = StandardScaler()
        scaler.fit(table.reshape(-1, 1))
        return scaler.transform(table.reshape(-1, 1))

In [6]:
# create custom dataset
train_ds = CustomDataset(train_df, train_df.columns[1:], train_df.columns[0])
test_ds = CustomDataset(test_df, test_df.columns[1:], test_df.columns[0])

In [7]:
batch_size = 16

# put custom dataset to dataloader
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

In [8]:
# get how many classes are there
class_ids = np.sort(train_df["label"].unique())
class_ids, len(class_ids)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int64),
 24)

In [9]:
# check shapes of dataloader
feature, label = next(iter(train_dl))
feature.shape, label.shape

(torch.Size([16, 1, 28, 28]), torch.Size([16]))

### 3.1. Create Model

In [10]:
class TinyVGG(nn.Module):

    def __init__(self):
        super().__init__()

        self.block_1 = nn.Sequential(
            nn.Conv2d(1, 10, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.block_2 = nn.Sequential(
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(10*7*7, len(class_ids)+1) # 7 * 7 because maxpool has been done twice which divides the shape of image by 2 twice
        )

    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x

model = TinyVGG().to(device)

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

### 3.2 Train Model

In [12]:
torch.manual_seed(20)

epochs = 5

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n------")
    
    # TRAINING
    train_loss, train_acc = 0, 0
    model.train()
    for batch, (X, y) in enumerate(train_dl):

        # forward pass
        train_pred = model(X)

        # metrics
        loss = loss_fn(train_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y, y_pred=train_pred.argmax(dim=1))

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # print metrics
    train_loss /= len(train_dl)
    train_acc /= len(train_dl)
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")

    
    # TESTING
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in test_dl:

            # forward pass
            test_pred = model(X)

            # metrics
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
        
        # print metrics
        test_loss /= len(test_dl)
        test_acc /= len(test_dl)
        print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
------
Train Loss: 1.9465 | Train Accuracy: 41.31%


 20%|██        | 1/5 [00:37<02:29, 37.48s/it]

Test Loss: 0.8295 | Test Accuracy: 76.92%
Epoch: 1
------
Train Loss: 0.1148 | Train Accuracy: 96.72%


 40%|████      | 2/5 [01:14<01:51, 37.18s/it]

...

Test Loss: 0.8459 | Test Accuracy: 83.09%
Epoch: 3
------
Train Loss: 0.0008 | Train Accuracy: 100.00%


 80%|████████  | 4/5 [02:27<00:36, 36.71s/it]

Test Loss: 0.8596 | Test Accuracy: 84.58%
Epoch: 4
------
Train Loss: 0.0004 | Train Accuracy: 100.00%


100%|██████████| 5/5 [03:06<00:00, 37.26s/it]

Test Loss: 0.8812 | Test Accuracy: 85.40%
